In [ ]:
project_path = "beautiful-pixel-formation-data-science/ds_rakuten"
run_names = 
load_wandb_runs(project_path, 

In [1]:
import wandb
import pandas as pd


def load_wandb_runs(
    project_path,
    run_names=None,
    states=("finished",),
    filters=None,
):
    """
    Charge les runs d'un projet Weights & Biases avec filtrage optionnel.

    Args:
        project_path (str):
            Chemin du projet W&B au format "entity/project".
        run_names (list[str] | None):
            Liste des noms de runs à récupérer (ex: ["camembert_texte brute"]).
            Si None, tous les runs sont chargés.
        states (tuple[str]):
            États des runs à inclure (par défaut ("finished",)).
        filters (dict | None):
            Filtres W&B avancés (ex: {"config.transformation": "numtok light"}).

    Returns:
        list[wandb.apis.public.Run]:
            Liste des runs W&B correspondants.
    """
    api = wandb.Api()
    runs = api.runs(project_path, filters=filters)

    selected_runs = []
    for run in runs:
        if states and run.state not in states:
            continue
        if run_names and run.name not in run_names:
            continue
        selected_runs.append(run)

    return selected_runs


In [ ]:
plot_f1_delta_by_class_sorted([0,1,1,1],[1,0,1,1], [1,0,1,1]

In [1]:
# en cours avec Céline



import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score


def plot_f1_delta_by_class_sorted(
    y_true
    y_pred_new,
    y_pred_ref,
    class_names=None,
    title="Δ F1-score par classe (référence - nouveau)",
    decimals=3,
):
    """
    Affiche un bar plot du delta de F1-score par classe entre deux modèles,
    trié par gain décroissant et coloré selon le signe du delta.

    Le delta est défini comme :
        F1(modèle de référence) - F1(nouveau modèle)

    Args:
        y_true (array-like):
            Labels réels.
        y_true_ref (array-like):
            Labels réels du modèle de référence.
        y_pred_ref (array-like):
            Prédictions du modèle de référence.
        class_names (list[str] | None):
            Noms des classes. Si None, indices numériques utilisés.
        title (str):
            Titre du graphique.
        decimals (int):
            Nombre de décimales affichées sur les barres.

    Returns:
        np.ndarray:
            Delta F1 par classe (référence - nouveau), trié.
    """
    # F1 par classe
    f1_new = f1_score(y_true, y_pred_new, average=None)
    f1_ref = f1_score(y_true, y_pred_ref, average=None)

    delta_f1 = f1_ref - f1_new

    if class_names is None:
        class_names = [f"Classe {i}" for i in range(len(delta_f1))]

    # Tri décroissant
    order = np.argsort(delta_f1)[::-1]
    delta_f1_sorted = delta_f1[order]
    class_names_sorted = np.array(class_names)[order]

    # Couleurs
    colors = ["tab:green" if v > 0 else "tab:red" for v in delta_f1_sorted]

    # Plot
    plt.figure(figsize=(13, 5))
    bars = plt.bar(class_names_sorted, delta_f1_sorted, color=colors)

    plt.axhline(0, linestyle="--")
    plt.ylabel("Δ F1-score")
    plt.title(title)
    plt.xticks(rotation=45, ha="right")

    # Valeurs sur les barres
    for bar, value in zip(bars, delta_f1_sorted):
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            value,
            f"{value:.{decimals}f}",
            ha="center",
            va="bottom" if value > 0 else "top",
            fontsize=9,
        )

    plt.tight_layout()
    plt.show()

    return delta_f1_sorted
